# 实验 4：完整怪物 Gridworld 中的多智能体对比分析

---

## 🎯 实验目的：

在包含以下复杂特性的 Gridworld 中：

- 怪物动态巡逻
- 钥匙与门机制
- 部分可观测性

对比多种智能体策略的表现，包括：

| 智能体 | 特点 |
|--------|------|
| 🎯 POMCP | 粒子滤波 + MCTS，适用于POMDP |
| 🧠 MCTS | 完全可观测MCTS（理论上性能差） |
| ⚡ Greedy | 每步向目标靠近 |
| 🎲 Random | 完全随机动作，作为基线 |

---

## 📊 评估指标：

- ✅ 成功率
- ✅ 平均奖励
- ✅ 平均步数
- ✅ 规划时间

---


In [ ]:
from core import MonsterGridworld, POMCPAgent, MCTSAgent, BaseAgent
from utils.base_experiment import BaseExperiment
from utils.metrics import plot_results
from utils.utils import set_seed
import random
import os


In [ ]:
class RandomAgent(BaseAgent):
    def act(self, obs):
        return random.choice(list(self.env.ACTIONS.keys()))


class GreedyAgent(BaseAgent):
    def act(self, obs):
        # 朝目标方向移动
        ax, ay = self.env.agent_pos
        gx, gy = self.env.goal_pos

        if gx > ax:
            return 1  # down
        if gx < ax:
            return 0  # up
        if gy > ay:
            return 3  # right
        if gy < ay:
            return 2  # left
        return random.choice(list(self.env.ACTIONS.keys()))


In [ ]:
set_seed(42)
os.makedirs("results", exist_ok=True)

agent_configs = [
    ("POMCP", POMCPAgent, {
        "belief_type": "monster",
        "n_simulations": 400,
        "max_depth": 25,
        "num_particles": 300,
    }),
    ("MCTS", MCTSAgent, {
        "n_simulations": 400,
        "max_depth": 25,
    }),
    ("Greedy", GreedyAgent, {}),
    ("Random", RandomAgent, {}),
]

N_EPISODES = 50

common_env_args = {
    "grid_size": (5, 5),
    "start_pos": (0, 0),
    "goal_pos": (4, 4),
    "obstacles": {(1, 3), (3, 1)},
}


In [ ]:
results = []
file_paths = []

for name, agent_cls, agent_args in agent_configs:
    output_file = f"results/exp3_monster_{name.lower()}.json"
    config = {
        "seed": 42,
        "num_episodes": N_EPISODES,
        "max_steps": 70,
        "output_file": output_file,
        "env_args": common_env_args,
        "agent_args": agent_args
    }

    print(f"🚀 Running agent: {name}")
    exp = BaseExperiment(MonsterGridworld, agent_cls, config)
    metrics = exp.run()
    results.append(metrics)
    file_paths.append(output_file)
    print(f"✅ {name} finished: {metrics}\n")


In [ ]:
labels = [name for name, _, _ in agent_configs]



## ✅ 实验总结：

| 智能体 | 表现特点 |
|--------|----------|
| 🎯 POMCP | 成功率最高，智能决策但耗时较大 |
| 🧠 MCTS | 无法应对部分可观测性，表现差 |
| ⚡ Greedy | 简单策略，在有钥匙和门的环境下容易失败 |
| 🎲 Random | 仅作基线，表现最差但时间最快 |

---

## 🧠 关键启发：

- POMDP 环境需要 **显式处理不确定性**，如粒子滤波
- 单靠 MCTS 无法处理观测模糊 → 信念建模至关重要
- Greedy 策略难以处理 **中间子目标**（拿钥匙、开门）

---

## 🔜 下一步扩展建议：

- ✅ 加入信念误差对比（真实位置 vs 粒子均值）
- ✅ 可视化轨迹或粒子动画，观察 agent 决策路径
- ✅ 比较更复杂 POMDP 算法（Q-MDP, DESPOT 等）

---
